# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
This notebook extracts and processes the data in the table of postcodes from the wikipedia page at: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

We ignore unused postcodes and merge all neighbourhoods with the same postcode into a single row, and assign the suburb name to the neighbourhood where one the neighbourhood has not been assigned a separate name.


### Import the packages we'll use

In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML

### Download the wiki page and convert into an object tree

In [29]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wiki_page = requests.get(url).text
    
soup = BeautifulSoup(wiki_page,'lxml')

### Find the table part of the page and extract the headers
3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [30]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#print(My_table)

ths = My_table.findAll('th')
headers=[]
for th in ths:
    headers.append(th.text.strip())
headers

['Postcode', 'Borough', 'Neighbourhood']

### Now add the data
Find all of the table rows containing data and build up a dictionary of postcodes, boroughs and neighbourhoods, filtering and appending neighbourhoods to existing postcodes as we go.
* 3.2 Only process the cells that have an assigned borough. _Ignore cells with a borough that is Not assigned._
* 3.3 More than one neighborhood can exist in one postal code area. _For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table._
* 3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. _So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park._

In [31]:
# Find all of the table rows to process
rows = My_table.findAll('tr')
postcode_list={}
# lose the first row as that's the headers that we have already processed
rows=rows[1:]
for row in rows:
    tds = row.findAll('td')
    postcode = tds[0].text.strip()
    borough = tds[1].text.strip()
    neigh = tds[2].text.strip()
    
    #3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
    if (neigh == "Not assigned"):
        neigh = borough

    ## 3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if (borough != 'Not assigned'):
        postcode_dict = {
            "Postcode": postcode, 
            "Borough": borough, 
            "Neighbourhood": neigh
        }
        ## 3.3 If we already have an entry for this postcode, then concatenate the neighbourhoods and updating the existing entry
        if (postcode in postcode_list):
            concatenated_neigh = postcode_list[postcode]["Neighbourhood"] + ", " + neigh
            postcode_dict.update({"Neighbourhood": concatenated_neigh})
            postcode_list[postcode].update(postcode_dict)
        else:
            postcode_list[postcode] = postcode_dict

### Convert our dictionary of postcode data into a dataframe
Show the first 12 entries.

In [32]:
df = pd.DataFrame(list(postcode_list.values()), columns=headers)
df.set_index('Postcode')
df.head(12)

,Postcode,Borough,Neighbourhood
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
1,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St..."
2,M2J,North York,"Fairview, Henry Farm, Oriole"
3,M9L,North York,Humber Summit
4,M1B,Scarborough,"Rouge, Malvern"
5,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
6,M6S,West Toronto,"Runnymede, Swansea"
7,M1J,Scarborough,Scarborough Village
8,M6J,West Toronto,"Little Portugal, Trinity"
9,M1W,Scarborough,"L'Amoreaux West, Steeles West"


# Now check the shape of the data frame
3.6 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [33]:
df.shape

(103, 3)

### Obtain co-ordinates for the suburbs
* 4.2 Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [60]:
#use geocoder or csv

import csv

csv_url = 'https://cocl.us/Geospatial_data'

with requests.Session() as s:
    download = s.get(csv_url)

    decoded_content = download.content.decode('utf-8')

    reader = csv.reader(decoded_content.splitlines(), delimiter=',')
    csv_data = list(reader)
            
latlong_df=pd.DataFrame(csv_data[1:], columns=['Postcode', 'Latitude', 'Longitude'])
latlong_df.set_index('Postcode')
latlong_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


### Update our dataframe with the co-ordinates of each postcode
e.g. 'M4N' should have co-ordinates: ('43.7280205', '-79.3887901')

In [59]:
df = pd.merge(df, latlong_df, how='outer', on='Postcode')
#convert the lat/long strings to numeric
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].apply(pd.to_numeric)
df


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
1,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
2,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
3,M9L,North York,Humber Summit,43.756303,-79.565963
4,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
5,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
6,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
7,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
8,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
9,M1W,Scarborough,"L'Amoreaux West, Steeles West",43.799525,-79.318389
